In [1]:
#!python -m spacy download en_core_web_lg
import json
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span
from spacy.matcher import PhraseMatcher
from spacy.pipeline import EntityRuler
from spacy.tokenizer import Tokenizer
from spacy import displacy, Language
import en_core_web_lg
# nlp = en_core_web_lg.load()
# nlp = spacy.load('en_core_web_lg')
import nltk
from nltk import Tree
from nltk import punkt
from nltk.tokenize import sent_tokenize
# nltk.download('punkt')
import dateparser
import re
from currencies import Currency
import glob
import os
import re
import pandas as pd
import string
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta, date

In [2]:
nlp = en_core_web_lg.load()
tickers = [ #ticker keywords
          "TSMC", "Taiwan Semiconductor", "\(TSM\)",
          "NVDA", "NVIDIA", "Nvidia", "NVidia",
          "ASML", "Advanced Semiconductor Materials International",
          "QRVO", "Qorvo",
          "RNECY", "Renesas",
          "ASX",
          "ATEYY", "Advantest", "ADVANTEST",
          "OIIM", "O2Micro",
          "\(RESN\)", "Resonant", "RESONANT",
          "INTT", "inTEST" , "inTest"
]
keywords = tuple(["target price", "price target", "price objective", "objective price"])

In [3]:
# Analyst & Organizations from TipRanks
path_global = "g:/My Drive/ETF/"
path = [f"{path_global}Analysts.xlsx",
        f"{path_global}Bloggers.xlsx"]
data_a = pd.read_excel(path[0], sheet_name="Sheet1")
data_b = pd.read_excel(path[1], sheet_name="Sheet1")
#data_b["name_"] = data_b["url"].str.replace("/analysts/", "").str.replace("-", " ")
data = pd.concat([data_a, data_b], ignore_index=True)
labels = []
tokenizer = Tokenizer(nlp.vocab)
for names in data["name"].dropna(how='all').unique():
    labels.append({"label": "ANALYST", "pattern": [{"ORTH": name} for name in [str(token) for token in tokenizer(names)]]})
labels.append({"label": "ANALYST", "pattern":"Analysts"})
labels.append({"label": "ANALYST", "pattern":"analysts"})
for names in data["worksFor"].dropna(how='all').unique():
    labels.append({"label": "ANALYST ORG", "pattern": [{"ORTH": name} for name in [str(token) for token in tokenizer(names)]]})

# Tickers
labels.extend([{"label": "ORG", "pattern": [{"ORTH": t} for t in [str(token) for token in tokenizer(ticker)]]} for ticker in tickers])

try:
  nlp.remove_pipe(name="entity_ruler")
except:
  pass
# ruler = EntityRuler(nlp, overwrite_ents=True)
# ruler = nlp.add_pipe("entity_ruler", overwrite_ents=True)
ruler = nlp.add_pipe("entity_ruler", before="ner")
# nlp.add_pipe(ruler, before='ner')
ruler.add_patterns(list(labels))

In [4]:
# Add Entities corrector to pipeline
class UpdateEntities(object):
    def __init__(self, nlp, label, mask, tail):
        from spacy.strings import StringStore
        if not label in nlp.vocab.strings:
          nlp.vocab.strings.add(label)
        self.label = nlp.vocab.strings[label]
        # self.label = label
        self.mask = mask
        self.tail = tail
    def __call__(self, doc):
        import re
        from spacy.tokens import Span
        if self.tail:
          tail_len = len(self.tail)+1
        else:
          tail_len = 0
        matches = [(m.start(), (m.end()-tail_len)) for m in re.finditer(self.mask, doc.text, re.IGNORECASE)] #m.group()
        ents = list(doc.ents)
        # for ent in ents:
        #     if ent.label_ == self.label:
        #         bool = True
        if matches:
          for match in matches:
            start, end = match
            indx = set()
            for i, token in enumerate(doc):
              if token.idx == start or (token.idx+len(token)) == end:
                # if i==(len(doc)-1):
                #   indx.add(i+1)
                # else:
                  indx.add(i)
            entity = Span(doc, min(indx), max(indx)+1, label=self.label)
            ents = tuple([ent for ent in ents if ent.start_char not in range(start,end)]) + (entity,)
        doc.ents = ents
        return doc

In [5]:
# Analyst ORG
try:
  nlp.remove_pipe(name="analyst_org")
except:
  pass

@Language.factory("analyst_org")
def new_component(nlp, name):
    return UpdateEntities(nlp, label="ANALYST ORG", mask= r"^[A-Z][A-Z\.a-z/]+\W+analyst", tail="analyst")
# nlp.add_pipe(analyst_org , name="analyst_org", before="ner")
nlp.add_pipe("analyst_org", before="ner", )

In [6]:
from currencies import Currency
# CURR = r"|".join([key for key in Currency.money_formats])
# REG = r"([A-Z]*\$\d+[.,]?\d*[^., ])|(\d+[.,]?\d+\s?(" + CURR + "|yen|euro){1})|((" + CURR + "|yen|euro){1}\s?((\d)+[.,]?\d+))|(\$\d+)"
#All codes & symbols
path_currencies = f"{path_global}currencies.json"
with open(path_currencies, encoding='utf-8') as file:
     symbols = json.load(file)
CURR_PREFIX = {key["symbol"] for key in symbols}
CURR_SUFFIX = {str(re.findall(r"\w+$", key["name"])[0]).lower() for key in symbols} | {str(re.findall(r"\w+$", key["name_plural"])[0]).lower() for key in symbols}
CURR_CODES = {key["code"] for key in symbols}
CURR_ALL = "|".join(CURR_PREFIX | CURR_SUFFIX | CURR_CODES)

# Mask for CURRENCY entity
REG = r"([A-Z]*\$\d+[.,]?\d*[^., ])|(\d+[.,]?\d+\s?(" + CURR_ALL + "){1})|((" + CURR_ALL + "){1}\s?((\d)+[.,]?\d+))|(\$\d+)"

currency_codes = pd.read_json(path_currencies)

In [7]:
@Language.factory("currency")
def new_component(nlp, name):
    return UpdateEntities(nlp, label="CURRENCY", mask= REG, tail=None)

try:
  nlp.remove_pipe(name="currency")
except:
  pass
# nlp.add_pipe(currency, name="currency", before="ner")
nlp.add_pipe("currency", before="ner")


In [8]:
# All the rules with objects
class AllRules(object):
    def __init__(self, nlp):
        pass
    def __call__(self, doc):

      # Matcher
      labels = ("RATING", "TARGET STRING", "ANALYST",  "ANALYST ORG", "TARGET PRICE", "OBJ", "VERB")
      ENTITIES = ["ORG", "NORP", "OBJ", "GPE", "LOC", "WORK_OF_ART"]
      def add_label(matcher, doc, i, matches):
        ents = list(doc.ents)
        try:
          match_id, start, end = matches[i]
          entity = Span(doc, start, end, label=label)
          doc.ents += (entity,)
        except:
            match_id, start, end = matches[i]
            for element in [start, end-1]:
              for j, ent in enumerate(doc.ents):
                #if ent.start_char == doc[element].idx and (ent.label_ not in labels or ent.label_=="RATING" and label=="OLD RATING"):
                  #ents[j] = Span(doc, ents[j].start, ents[j].end, label=label)
                if any([item in range(ent.start, ent.end) for item in range(start, end)]) and (ent.label_ not in labels or (ent.label_=="RATING" and label=="OLD RATING")):
                  entity = Span(doc, ent.start, ent.end, label=label)
                  ents =  tuple([e for e in ents if e.start != ent.start]) + (entity,)
            doc.ents = ents

      def add_match(matcher, label, pattern, on_match):
          matcher.add(label, pattern, on_match=on_match)


      # Simple ratigs
      matcher_rating = Matcher(nlp.vocab)
      ratings = ["sell", "moderate-sell", "underweight", "under-weight", "hold", "holds", "neutral",
                "outperform", "moderate-buy", "overweight", "over-weight", "accumulate", "add", "buy",
                "strong-buy", "bullish", "underperform", "positive", "negative", "reduce"]
      pattern = [[{"ORTH": '"'}, {"LOWER": rating}, {'ORTH': '"'}] for rating in ratings] + \
                [[{"ORTH": '“'}, {"LOWER": rating}, {'ORTH': '”'}] for rating in ratings] + \
                [[{"ORTH": 'rating', "OP": "+"}, {"LOWER": rating}] for rating in ratings] + \
                [[{"LOWER": rating}, {"ORTH": 'rating', "OP": "+"}] for rating in ratings] + \
                [[{"ORTH": rating.title()}] for rating in ratings]
      add_match(matcher_rating, "RATING", pattern, add_label)

      # Complex rating
      matcher_phrase = PhraseMatcher(nlp.vocab)
      phrases = ["strong sell", "equal weigh", "equal weight", "moderate sell", "strong buy", "weak hold", "market perform", "sector perform"]
      pattern = [nlp.make_doc(phrase) for phrase in phrases] + \
                [nlp.make_doc(phrase.title()) for phrase in phrases]
      add_match(matcher_phrase, "RATING", pattern, add_label)

      # Old rating
      matcher_old_rating = Matcher(nlp.vocab)
      pattern = [
                    [
                    {"TEXT": "from"},
                    {"ENT_TYPE": "RATING"}
                    ]
                  ]
      add_match(matcher_old_rating, "RATING OLD", pattern, add_label)

      # Target string
      matcher_target = PhraseMatcher(nlp.vocab)
      prices = ["target price", "price target", "price objective", "objective price", "price targets"]
      pattern = [nlp.make_doc(phrase) for phrase in prices]
      add_match(matcher_target, "TARGET STRING", pattern, add_label)

      # Analyst rematch
      matcher_analyst = Matcher(nlp.vocab)
      pattern = [
                {"LEMMA": {"IN": ["analyst", "Analyst"]}}, {"ENT_TYPE": "PERSON"}
      ]
      add_match(matcher_analyst, "ANALYST", [pattern], add_label)

      # Analyst ORG
      matcher_analyst_org = Matcher(nlp.vocab)
      pattern = [
                [
                 {"ENT_TYPE": {"IN": ["ORG", "NORP", "OBJ", "PERSON"]}},
                {"LEMMA": "analyst"}
                ],
                [
                 {"ENT_TYPE": {"IN": ["ORG", "NORP", "OBJ", "PERSON","GPE"]}},
                 {"TEXT": "analyst"},
                ],
                # [
                #  {"POS": "NOUN"},
                #  {"POS": "NOUN", "OP": "*"},
                #  {"LEMMA": "analyst"}
                # ],
      ]
      add_match(matcher_analyst_org , "ANALYST ORG", pattern, add_label)

      # Target price
      matcher_price = Matcher(nlp.vocab)
      pattern = [
                [
                  {"TEXT": "rating"},
                  {"TEXT": "a"},
                  {"ENT_TYPE": "CURRENCY"}
                ],
                [
                  {"ENT_TYPE": "TARGET STRING"},
                  {"POS": "ADP", "OP": "*"},
                  {"ENT_TYPE": {"IN": ["ORG", "NORP", "OBJ", "GPE"]},"OP": "*" },
                  {"POS": "ADP", "OP": "*"},
                  {"POS": "SYM", "OP": "*"},
                  {"ENT_TYPE": {"IN": ["CURRENCY", "MONEY"]}}
                ],
                [
                  {"ENT_TYPE": "TARGET STRING"},
                  {"POS": "ADP", "OP": "*"},
                  {"POS": "DET", "OP": "*"},
                  {"POS": "NOUN", "OP": "*"},
                  {"POS": "ADP", "OP": "*"},
                  {"POS": "SYM", "OP": "*"},
                  {"ENT_TYPE": {"IN": ["CURRENCY", "MONEY"]}}
                ],
                [
                  {"ENT_TYPE": {"IN": ENTITIES}},
                  {"TEXT": "shares"},
                  {"POS": "ADP", "OP": "*"},
                  {"POS": "SYM", "OP": "*"},
                  {"ENT_TYPE": {"IN": ["CURRENCY"]}}
                ],
                [
                  {"ENT_TYPE": {"IN": ENTITIES}},
                  {"TEXT": "to"},
                  {"TEXT": "a"},
                  {"ENT_TYPE": {"IN": ["CURRENCY"]}}
                ],
                [
                  {"ENT_TYPE": {"IN": ["CURRENCY", "MONEY", "CARDINAL"]}},
                  {"ENT_TYPE": "TARGET STRING"}
                ],
                [
                  {"ENT_TYPE": {"IN": ["RATING"]}},
                  {"POS": {"IN": ["NOUN", "CCONJ"]}},
                  {"POS": {"IN": ["NOUN", "CCONJ"]}},
                  {"ENT_TYPE": {"IN": ["CURRENCY"]}}
                ],
                [
                  {"TEXT": "with"},
                  {"TEXT": "a"},
                  {"ENT_TYPE": "CURRENCY"}
                ],
                [
                  {"ENT_TYPE": {"IN": ["CURRENCY"]}},
                  {"TEXT": "from"}
                ],
                [
                  {"TEXT": "with"},
                  {"TEXT": "a"},
                  {"ENT_TYPE": "CURRENCY"}
                ],
                [
                  {"POS": "PUNCT"},
                  {"TEXT": "to"},
                  {"ENT_TYPE": "CURRENCY"}
                ],
                [
                  {"ENT_TYPE": "CURRENCY"},
                  {"ENT_TYPE": "TARGET STRING"}
                ],
                [
                  {"ENT_TYPE": "RATING"},
                  {"TEXT": "and"},
                  {"TEXT": "a", "OP": "*"},
                  {"ENT_TYPE": "CURRENCY"}
                ],
                [
                  {"ORTH": "to"},
                  {"ENT_TYPE": "CURRENCY"},
                  {"ORTH": "from"}
                ],
                [
                  {"ENT_TYPE": "TARGET STRING"},
                  {"TEXT": "on"},
                  {"POS": "PROPN"},
                  {"TEXT": "to"},
                  {"ENT_TYPE": "CURRENCY"},
                  {"ORTH": "from", "OP": "*"}
                ],
                [
                {"LOWER": "street"},
                {"ORTH": "-"},
                {"ORTH": "high"},
                {"ENT_TYPE": "CURRENCY"},
                ],
                [
                  {"ENT_TYPE": "OBJ"},
                  {"ORTH": "(", "OP": "*"},
                  {"ENT_TYPE": {"IN": ENTITIES}, "OP": "*"},
                  {"ORTH": ")", "OP": "*"},
                  {"ORTH": "shares", "OP": "*"},
                  {"ORTH": "to"},
                  {"ENT_TYPE": "CURRENCY"},
                ],
                ]
      add_match(matcher_price, "TARGET PRICE", pattern, add_label)

      # OBJ
      matcher_org = Matcher(nlp.vocab)
      pattern = [
                  [
                    {"ENT_TYPE": "TARGET STRING"},
                    {"POS": "ADP", "OP": "*"},
                    {"ENT_TYPE": {"IN": ENTITIES}}
                  ],
                  [
                    {"ENT_TYPE": "ANALYST"},
                    {"POS": "VERB"},
                    {"ENT_TYPE": {"IN": ENTITIES}}
                  ],
                  [
                    {"ENT_TYPE": "TARGET STRING"},
                    {"POS": "ADP"},
                    {"ENT_TYPE": {"IN": ENTITIES}}
                  ],
                  [
                    {"ENT_TYPE": {"IN": ENTITIES}},
                    {"POS": "ADP"},
                    {"POS": "DET"},
                   {"ENT_TYPE": {"IN": ["RATING", "OLD RATING"]}}
                  ],
                  [
                    {"ENT_TYPE": "TARGET STRING"},
                    {"TEXT": "on"},
                    {"POS": "PROPN"}
                  ],
                  [
                   {"ENT_TYPE": "ANALYST"},
                   {"POS": "VERB"},
                   {"ENT_TYPE": {"IN": ENTITIES}}
                  ],
                  [
                   {"POS": "PROPN"},
                   {"POS": "ADP"},
                   {"POS": "DET"},
                   {"ENT_TYPE": {"IN": ["RATING", "OLD RATING"]}}
                  ],
                   [
                   {"ENT_TYPE": {"IN": ["ORG", "NORP", "PERSON", "GPE", "LOC"]}},
                   {"TEXT": "with"},
                   {"TEXT": "a"},
                   {"ENT_TYPE": "RATING"}
                  ],
                  [
                   {"ENT_TYPE": "RATING"},
                   {"ORTH": "-", "OP": "*"},
                   {"LOWER": "rated", "OP": "*"},
                   {"ENT_TYPE": {"IN": ENTITIES}}
                 ],
                ]
      add_match(matcher_org, "OBJ", pattern, add_label)

      # VERB for ratings
      matcher_verb = Matcher(nlp.vocab)
      pattern = [
                  [
                   {"ENT_TYPE": {"IN": ["ANALYST", "ANALYST ORG"]}},
                   {"POS": "VERB"},
                  ]
                ]
      def add_label_verb(matcher, doc, i, matches):
          match_id, start, end = matches[i]
          entity = Span(doc, end-1, end, label=label)
          doc.ents += (entity,)

      add_match(matcher_verb, "VERB", pattern, add_label_verb)

      #traing of rules
      label = "RATING"
      matcher_rating(doc)
      matcher_phrase(doc)
      label = "OLD RATING"
      matcher_old_rating(doc)
      label = "TARGET STRING"
      matcher_target(doc)
      label = "OBJ"
      matcher_org(doc)
      label = "ANALYST"
      matcher_analyst(doc)
      label = "TARGET PRICE"
      matcher_price(doc)
      label = "ANALYST ORG"
      matcher_analyst_org(doc)
      label = "OBJ"
      matcher_org(doc)
      label = "TARGET PRICE"
      matcher_price(doc)
      label = "VERB"
      matcher_verb(doc)
      return doc

# Ad to the pipe
try:
  nlp.remove_pipe(name="all_rules")
except:
  pass

@Language.factory("all_rules")
def new_component(nlp, name):
    return AllRules(nlp)

# nlp.add_pipe(all_rules, name="all_rules", last=True)
nlp.add_pipe("all_rules", last=True)
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'entity_ruler',
 'analyst_org',
 'currency',
 'ner',
 'all_rules']

In [9]:
one = """BofA/Merrill analyst Vivek Arya double-upgraded Skyworks (SWKS) and Qorvo (QRVO) to Buy from Underperform, also raising his price targets to $122 from $92 and $130 from $80 respectively."""
two = """Societe Generale started ASML with a Buy rating and 220 euro price target."""

In [10]:
text = "Piper Sandler analyst Harsh Kumar raised the firm's price target on Nvidia to $220 from $172.50 and reiterates an Overweight rating on the shares."
# for element in ["currency", "all_rules", 'entity_ruler','analyst_org']:
#     nlp.disable_pipe(element)
doc = nlp(text)
# label = "TEST"
# matcher_test(doc)
displacy.render(doc, style="ent", jupyter=True)

In [11]:
path = f"{path_global}2017-2021/"

In [12]:
def filter_data(df, column, key):
    return df.loc[df[column].str.contains("|".join(key), case=False, regex=True, na=False)].reset_index(drop=True)
def relevant_data(data):
    dataset = filter_data(data, "headline", tickers)
    dataset["keys"] = dataset.apply(lambda row: "; ".join([match.group() for match in re.finditer("|".join(tickers), row["headline"], re.IGNORECASE)]), axis=1)
    dataset["date"] = pd.to_datetime(dataset["date"], format="%d.%m.%Y", errors="coerce")
    dataset = dataset[dataset["date"] > "2016-12-31"]
    dataset.dropna(inplace=True)
    dataset["sentence"] = dataset.apply(lambda row: sent_tokenize(row["body"])[0], axis=1)
    relevant = filter_data(dataset, "sentence", ["price target", "target price"])
    relevant[["date", "URL", "headline", "body", "sentence", "keys"]].to_excel(f"{path}TheFly_tokenize.xlsx", index=False)

In [13]:
# dataset = pd.read_excel(f"{path}TheFly_tokenize.xlsx", sheet_name="Sheet1")
# dataset[["date", "headline", "sentence", "keys"]]

In [14]:
# COLUMNS = ["OBJ", "ANALYST ORG", "ANALYST", "RATING", "TARGET PRICE"]
# final = pd.DataFrame(columns = dataset.columns.tolist() + COLUMNS)
# j=0
#
# def ent_iteration():
#     for ent in doc.ents:
#         columns = COLUMNS
#         if ent.label_ in columns:
#             final.loc[j, ent.label_] = ent
#             ENTS.append(ent.label_)
#             columns.remove(ent.label_)
# #iteration
# for i, text in enumerate(dataset["sentence"]):
#     doc = nlp(text)
#     ENTS = []
#     ent_iteration()
#     if "TARGET PRICE" not in ENTS:
#         doc =nlp(text + " " + dataset.loc[i,"headline"])
#         ent_iteration()
#     for column in dataset.columns.tolist():
#         final.loc[j, column] = dataset.loc[i, column]
#     j+=1
# final.to_excel(path + "/ratings.xlsx")
# final

In [15]:
def clean_currency_code(string):
    string = re.sub(r"[\d\., ]", "", string)
    code = currency_codes[currency_codes["symbol"]==string]["code"]
    if code.empty:
        return currency_codes[currency_codes["name"].str.contains(string, case=False, regex=True)]["code"].values[0]
    else:
        return code.values[0]


def clean_currency(string):
    return float("".join(re.findall(r"[0-9\.]+", string)))

In [16]:
pd.options.display.float_format = '{:,.2f}'.format
ratings = pd.read_excel(f"{path_global}ratings - Copy.xlsx", parse_dates=True)
ratings["CURRENCY"] = ratings["TARGET PRICE"]
ratings["CURRENCY"] = ratings.apply(lambda row: clean_currency_code(row["CURRENCY"]), axis=1)
ratings["TARGET PRICE"] = ratings.apply(lambda row: clean_currency(row["TARGET PRICE"]), axis=1)
ratings["CURRENCY"].unique().tolist()
ratings.to_excel(f"{path_global}ratings.xlsx", index=False)

In [17]:
from datetime import date, datetime
pd.options.display.float_format = '{:,.2f}'.format
from currency_converter import CurrencyConverter
# from forex_python.converter import CurrencyRates
converter = CurrencyConverter()
PASS= ["USD"]
for i, price in enumerate(ratings["TARGET PRICE"]):
  currency= ratings.loc[i,"CURRENCY"]
  date = ratings.loc[i,"date"]
  try:
    if currency not in PASS:
        ratings.loc[i, "PRICE"] = converter.convert(price, currency, 'USD', date=date)
    else:
        ratings.loc[i,"PRICE"] = price
  except:
    rate = yf.Ticker(f"{currency}USD=X").history(period="max")
    ratings.loc[i,"PRICE"] = rate.loc[date, "Close"]*price

# ratings.set_index(keys="date", inplace=True)
# ratings.to_excel(f"{path_global}ratings_clean.xlsx", index=False)
ratings[ratings["CURRENCY"]!="USD"]

,date,URL,headline,body,sentence,OBJ,ANALYST ORG,ANALYST,RATING,TARGET PRICE,CURRENCY,PRICE
32,2021-07-12,https://thefly.com/onthefly.php?id=3333101,Advantest downgraded to Neutral from Buy at Go...,Goldman Sachs analyst Shuhei Nakamura downgrad...,Goldman Sachs analyst Shuhei Nakamura downgrad...,Advantest,Goldman Sachs,Shuhei Nakamura,Neutral,"10,000.00",JPY,90.79
45,2021-06-21,https://thefly.com/onthefly.php?id=3323414,TSMC downgraded to Equal Weight from Overweigh...,"On June 18, Morgan Stanley analyst Charlie Cha...","On June 18, Morgan Stanley analyst Charlie Cha...",TSMC,Morgan Stanley,Charlie Chan,Equal Weight,580.00,TWD,20.88
86,2021-04-07,https://thefly.com/onthefly.php?id=3278300,Advantest downgraded to Buy from Conviction Bu...,Goldman Sachs analyst Shuhei Nakamura removed ...,Goldman Sachs analyst Shuhei Nakamura removed ...,Advantest,Goldman Sachs,Shuhei Nakamura,Buy,"12,500.00",JPY,113.78
100,2021-02-24,https://thefly.com/onthefly.php?id=3253445,Advantest downgraded to Neutral from Buy at Mi...,Mizuho analyst Yoshitsugu Yamamoto downgraded ...,Mizuho analyst Yoshitsugu Yamamoto downgraded ...,Advantest,Mizuho,Yoshitsugu Yamamoto,Neutral,"10,000.00",JPY,94.38
119,2019-07-08,https://thefly.com/onthefly.php?id=2930236,ASML initiated with a Buy at Societe Generale,Societe Generale started ASML with a Buy ratin...,Societe Generale started ASML with a Buy ratin...,ASML,Societe Generale,NaN,Buy,220.00,EUR,246.73
...,...,...,...,...,...,...,...,...,...,...,...,...
648,2021-07-22,https://thefly.com/onthefly.php?id=3340257,ASML price target raised to EUR 800 from EUR 7...,Barclays analyst Andrew Gardiner raised the fi...,Barclays analyst Andrew Gardiner raised the fi...,ASML,Barclays,Andrew Gardiner,Overweight,800.00,EUR,942.00
649,2021-09-24,https://thefly.com/onthefly.php?id=3377271,ASML price target raised to EUR 800 from EUR 7...,Morgan Stanley analyst Dominik Olszewski raise...,Morgan Stanley analyst Dominik Olszewski raise...,ASML,Morgan Stanley,Dominik Olszewski,Overweight,800.00,EUR,937.52
650,2021-09-08,https://thefly.com/onthefly.php?id=3368801,ASML price target raised to EUR 845 from EUR 6...,Berenberg analyst Tammy Qiu raised the firm's ...,Berenberg analyst Tammy Qiu raised the firm's ...,ASML,Berenberg,Tammy Qiu,Buy,845.00,EUR,999.38
653,2020-03-11,https://thefly.com/onthefly.php?id=3049378,JPMorgan says recession would mean more downsi...,JPMorgan analyst Sandeep Deshpande maintained ...,JPMorgan analyst Sandeep Deshpande maintained ...,ASML,JPMorgan,Sandeep Deshpande,Overweight,280.00,EUR,317.41


In [18]:
TICKERS = {
          "TSM": "TSMC",
          "NVDA":  "Nvidia",
          "ASML": "ASML",
          "QRVO": "Qorvo",
          "RNECY": "Renesas",
          "ASX": "ASX",
          "ATEYY": "Advantest",
          "RESN": "Resonant",
          "INTT": "inTEST",
          "OIIM": "O2Micro",
}

In [19]:
RATINGS_MEANING= {
                "buy": 1,
                "strong-buy": 1,
                "moderate-buy": 1,
                "strong buy": 1,
                "speculative buy": 1,
                "overweight": 1,
                "over-weight": 1,
                "outperform": 1,
                "accumulate": 1,
                "add": 1,
                "bullish": 1,
                "positive": 1,
                "top pick": 1,
                "focus buy": 1,
                "action list buy" : 1,

                "hold": 0,
                "holds": 0,
                "neutral": 0,
                "market perform": 0,
                "sector perform":0,
                "peer-preform": 0,
                "equal weight": 0,
                "equal weigh": 0,

                "sell": -1,
                "negative": -1,
                "reduce": -1,
                "strong sell": -1,
                "moderate-sell": -1,
                "moderate sell": -1,
                "underperform": -1,
                "underweight": -1,
                "under-weight": -1,
                "focus reduce": -1,
                "weak hold": -1,
                "bearish": -1,
                }
# VERBS_RATING =  {
#                  'increased': 1,
#                  'lowered': -1,
#                  'lowers': -1,
#                  'raised': 1,
#                  # 'reiterated': 0,
#                  'removed': -1,
#                 }

In [20]:
ratings["rating"]=ratings["RATING"].str.lower()
ratings  = ratings.replace({"rating": RATINGS_MEANING})

ratings['buy'] = ratings.apply(lambda x: x["PRICE"] if x["rating"]>0 else (None), axis=1)
ratings['sell'] = ratings.apply(lambda x: x["PRICE"] if x["rating"]<0 else (None), axis=1)
ratings['hold'] = ratings.apply(lambda x: x["PRICE"] if x["rating"]==0 else (None), axis=1)
ratings["USD"] = ratings.apply(lambda x: x["PRICE"] if np.isnan(x["rating"]) else (None), axis=1)

In [21]:
# import numpy as np
# for i, text in enumerate(ratings["sentence"]):
#     if np.isnan(ratings.loc[i, "rating"]):
#         doc = nlp(text)
#         verb = [ent.text for ent in doc.ents if ent.label_=="VERB"]
#         if verb:
#             ratings.loc[i, "rating"] = verb[0]

In [22]:
ratings.to_excel(f"{path_global}ratings.xlsx", index=False)
# ratings = ratings.replace({"rating": VERBS_RATING})
# ratings["rating"].fillna(0, inplace=True)
# ratings["rating"].unique()

In [23]:
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
from dash.dependencies import Input, Output
import dash_html_components as html

<ipython-input-23-33a8aa324fac>:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
<ipython-input-23-33a8aa324fac>:6: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [24]:
from pandas_datareader import data as web

app = dash.Dash('Hello World')

app.layout = html.Div([
    dcc.Dropdown(
        id='my-dropdown',
        options=[
            {'label': 'NVIDIA', 'value': 'NVDA'},
        ],
        value='NVDA'
    ),
    dcc.Graph(id='my-graph')
], style={'width': '500'})

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])
def update_graph(selected_dropdown_value):
    unit = yf.Ticker(selected_dropdown_value).history (
          start="2017-01-01",
          end="2021-09-30",
          interval='1d')
    return {
        'data': [{
            'x': unit.index,
            'y': unit.Close
        }],
        'layout': {'margin': {'l': 40, 'r': 0, 't': 20, 'b': 30}}
    }

app.css.append_css({'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'})

if __name__ == '__main__':
    app.run_server()



Dash is running on http://127.0.0.1:8050/

 * Serving Flask app 'Hello World' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
c:\users\eugene\appdata\local\programs\python\python39\lib\site-packages\dash\resources.py:63: UserWarning: You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources
  warnings.warn(
127.0.0.1 - - [02/Nov/2021 11:14:54] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2021 11:14:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2021 11:14:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2021 11:14:55] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2021 11:14:55] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Nov/2021 11:14:55] "GET /_dash-component-suites/dash/

In [ ]:
ticker = "TSM"
dataset = ratings[ratings["OBJ"] == TICKERS[ticker]]

unit = yf.Ticker(ticker).history (
      start="2017-01-01",
      end="2021-09-30",
      interval='1d')
fig = go.Figure()

fig.add_trace(go.Scatter(x=unit.index, y=unit["Close"],))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset["USD"], mode="markers"))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset["buy"], mode="markers"))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset["sell"], mode="markers"))
fig.add_trace(go.Scatter(x=dataset.index, y=dataset["hold"], mode="markers"))
fig.show()







